<a href="https://colab.research.google.com/github/Brownspite/CSE111/blob/main/model_of_422.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, fbeta_score
from seaborn import heatmap
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import plotly.express as px

dataset = pd.read_csv("/content/software_quality_dataset.csv")
dataset.shape

dataset.head()

dataset.info()

dataset.isnull().sum()

feature_data_types = {}
for col in dataset.columns:
    if col == 'Quality_Label':
        continue
    unique_values = dataset[col].unique()
    if pd.api.types.is_numeric_dtype(dataset[col]):
        feature_data_types[col] = "Quantitative"
    else:
        feature_data_types[col] = "Categorical"

target_variable_unique_values = dataset['Quality_Label'].unique() # Changed df to dataset
print(f"\nFeature Data Types:\n{feature_data_types}")
print(f"\nUnique values of the target variable 'Quality_Label': {target_variable_unique_values}")

# Encode the categorical features
le = LabelEncoder()
dataset['Has_Unit_Tests'] = le.fit_transform(dataset['Has_Unit_Tests'])
dataset['Quality_Label'] = le.fit_transform(dataset['Quality_Label'])

# Calculate the correlation matrix
correlation_matrix = dataset.corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Features')
plt.show()
# Analyze the correlation results
print("Correlation with Quality_Label:")
print(correlation_matrix['Quality_Label'].sort_values(ascending=False))

# Analyze the target variable for class imbalance
quality_counts = dataset['Quality_Label'].value_counts()
print("\nQuality Label Counts:")
print(quality_counts)

plt.figure(figsize=(8, 6))
quality_counts.plot(kind='bar', color=['skyblue', 'salmon', 'lightgreen'])
plt.title('Distribution of Quality Labels')
plt.xlabel('Quality Label')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

##Selecting numerical features
numerical_data = dataset.select_dtypes(include='number')

#append the features of numerical_data to list
numerical_features=numerical_data.columns.tolist()

print(f'There are {len(numerical_features)} numerical features:', '\n')
print(numerical_features)





#Selecting categoricalfeatures
categorical_data=dataset.select_dtypes(include= 'object')

#append the features of categorical_data to list
categorical_features=categorical_data.columns.tolist()

print(f'There are {len(categorical_features)} numerical features:', '\n')
print(categorical_features)

numerical_data.nunique()

# Impute missing values using the median for numerical features
for col in ['Lines_of_Code', 'Code_Churn', 'Comment_Density']:
    dataset[col].fillna(dataset[col].median(), inplace=True) # Changed df to dataset

# Check if 'Has_Unit_Tests' column exists before one-hot encoding
if 'Has_Unit_Tests' in dataset.columns:
    # One-hot encode the 'Has_Unit_Tests' column if it exists
    dataset = pd.get_dummies(dataset, columns=['Has_Unit_Tests'], prefix='Has_Unit_Tests') # Changed df to dataset

# Verify data cleaning
print("Number of missing values after imputation:")
print(dataset.isnull().sum()) # Changed df to dataset

print("\nFirst few rows of the cleaned data:")
display(dataset.head()) # Changed df to dataset

dataset.isnull().sum()

dataset = pd.get_dummies(dataset, columns=['Has_Unit_Tests'], prefix='Has_Unit_Tests')

scaler= StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

quality_counts = dataset['Quality_Label'].value_counts()
print("\nQuality Label Counts:")
print(quality_counts)

plt.figure(figsize=(8, 6))
quality_counts.plot(kind='bar', color=['skyblue', 'salmon', 'lightgreen'])
plt.title('Distribution of Quality Labels')
plt.xlabel('Quality Label')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

print("\nPercentage of each class:")
print(quality_counts / len(dataset) * 100)



from sklearn.model_selection import train_test_split

# Assuming your preprocessed dataset is stored in a pandas DataFrame called 'dataset'
X = dataset.drop('Quality_Label', axis=1)  # Features
y = dataset['Quality_Label']  # Target variable

# Stratified splitting with 70% for training and 30% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Create and train the KNN model with increased n_neighbors
knn_model = KNeighborsClassifier(n_neighbors=5)  # Increase n_neighbors to 5 (or another value)
knn_model.fit(X_train, y_train)

# Make predictions on the test set
knn_pred = knn_model.predict(X_test)

# Evaluate the model
knn_training_accuracy = accuracy_score(y_train, knn_model.predict(X_train))
knn_test_accuracy = accuracy_score(y_test, knn_pred)

print(f'KNN Training Accuracy: {knn_training_accuracy}')
print(f'KNN Testing Accuracy: {knn_test_accuracy}')

# Get precision, recall, and F1-score
report = classification_report(y_test, knn_pred)
print(report)

precision, recall, f1, support = precision_recall_fscore_support(y_test, knn_pred, pos_label=1)
knn_precision = precision[1]
knn_recall = recall[1]

print('KNN Precision:', knn_precision)
print('KNN Recall:', knn_recall)

# Create and train the Decision Tree model
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Make predictions on the test set
dt_pred = decision_tree_model.predict(X_test)

# Evaluate the model
dt_training_accuracy = accuracy_score(y_train, decision_tree_model.predict(X_train))
dt_test_accuracy = accuracy_score(y_test, dt_pred)

print(f'Decision Tree Training Accuracy: {dt_training_accuracy}')
print(f'Decision Tree Testing Accuracy: {dt_test_accuracy}')

# Get precision, recall, and F1-score
report = classification_report(y_test, dt_pred)
print(report)

precision, recall, f1, support = precision_recall_fscore_support(y_test, dt_pred, pos_label=1)
dt_precision = precision[1]
dt_recall = recall[1]

print('Decision Tree Precision:', dt_precision)
print('Decision Tree Recall:', dt_recall)

# Define the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer with 64 units and ReLU activation
    keras.layers.Dense(32, activation='relu'),  # Hidden layer with 32 units and ReLU activation
    keras.layers.Dense(1, activation='sigmoid')  # Output layer with 1 unit and sigmoid activation (for binary classification)
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use binary_crossentropy for binary classification
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)  # Adjust epochs and batch_size as needed

# Make predictions on the test set
nn_pred_probs = model.predict(X_test)
nn_pred = (nn_pred_probs > 0.5).astype(int)  # Convert probabilities to class labels (0 or 1)

# Evaluate the model
nn_training_accuracy = accuracy_score(y_train, (model.predict(X_train) > 0.5).astype(int))
nn_test_accuracy = accuracy_score(y_test, nn_pred)

print(f'Neural Network Training Accuracy: {nn_training_accuracy}')
print(f'Neural Network Testing Accuracy: {nn_test_accuracy}')

# Get precision, recall, and F1-score
report = classification_report(y_test, nn_pred)
print(report)

precision, recall, f1, support = precision_recall_fscore_support(y_test, nn_pred, pos_label=1)
nn_precision = precision[1]
nn_recall = recall[1]

print('Neural Network Precision:', nn_precision)
print('Neural Network Recall:', nn_recall)

models = ['KNN', 'Decision Tree', 'Neural Network']
accuracies = [knn_test_accuracy, dt_test_accuracy, nn_test_accuracy]

plt.figure(figsize=(8, 6))
plt.bar(models, accuracies, color=['blue', 'green', 'red'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)  # Set y-axis limits for better visualization
plt.show()

# 2. Precision-Recall Comparison
plt.figure(figsize=(8, 6))
plt.bar(models, [knn_precision, dt_precision, nn_precision], color=['blue', 'green', 'red'], label='Precision')
plt.bar(models, [knn_recall, dt_recall, nn_recall], color=['lightblue', 'lightgreen', 'lightcoral'], label='Recall', alpha=0.7)  # Added alpha for transparency
plt.title('Precision-Recall Comparison')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.legend()
plt.show()

knn_cm = confusion_matrix(y_test, knn_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(knn_cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('KNN Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Decision Tree Confusion Matrix
dt_cm = confusion_matrix(y_test, dt_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(dt_cm, annot=True, fmt='d', cmap='Greens', cbar=False)
plt.title('Decision Tree Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Neural Network Confusion Matrix
nn_cm = confusion_matrix(y_test, nn_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(nn_cm, annot=True, fmt='d', cmap='Reds', cbar=False)
plt.title('Neural Network Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Predict using each model
knn_pred = knn_model.predict(X_test)
dt_pred = decision_tree_model.predict(X_test)
nn_pred = np.argmax(model.predict(X_test), axis=1) # Changed neural_network_model to model

# Assuming y_test is your original target variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_test_encoded = le.fit_transform(y_test)  # Encode y_test if it's not already encoded


# Function to evaluate a model and print results
def evaluate_model(model_name, y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')

    print(f"--- {model_name} ---")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")


# Evaluate each model
evaluate_model("KNN", y_test_encoded, knn_pred)
evaluate_model("Decision Tree", y_test_encoded, dt_pred)
evaluate_model("Neural Network", y_test_encoded, nn_pred)

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

!pip install imblearn
from imblearn.under_sampling import RandomUnderSampler # Import RandomUnderSampler
# --- Undersampling ---
# Instantiate RandomUnderSampler
undersampler = RandomUnderSampler(random_state=42)

# Resample the data
X_undersampled, y_undersampled = undersampler.fit_resample(X, y)

# --- Oversampling (SMOTE) ---
# Instantiate SMOTE
smote = SMOTE(random_state=42)

# Resample the data
X_oversampled, y_oversampled = smote.fit_resample(X, y)


# --- Model Training and Evaluation (for each sampling technique) ---
def train_and_evaluate_model(X_train, y_train, X_test, y_test, model_name, model):
    """Trains and evaluates a given model."""
    model.fit(X_train, y_train)

    # Get predicted probabilities for Neural Network
    if model_name == "Neural Network":
        y_pred_probs = model.predict(X_test)
        y_pred = (y_pred_probs > 0.5).astype(int)  # Convert probabilities to class labels (0 or 1)
    else:
        y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    precision, recall, _, _ = precision_recall_fscore_support(y_test, y_pred, pos_label=1)

    print(f"--- {model_name} ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision[1]:.4f}")
    print(f"Recall: {recall[1]:.4f}")
    print(report)
    print("\n")

    return accuracy, precision[1], recall[1]  # Return metrics for comparison

# --- Original Data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# --- Undersampled Data ---
X_train_under, X_test_under, y_train_under, y_test_under = train_test_split(
    X_undersampled, y_undersampled, test_size=0.3, random_state=42, stratify=y_undersampled
)
X_train_under = scaler.fit_transform(X_train_under)
X_test_under = scaler.transform(X_test_under)


# --- Oversampled Data ---
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(
    X_oversampled, y_oversampled, test_size=0.3, random_state=42, stratify=y_oversampled
)
X_train_over = scaler.fit_transform(X_train_over)
X_test_over = scaler.transform(X_test_over)

# --- Model Initialization ---
knn_model = KNeighborsClassifier(n_neighbors=5)
decision_tree_model = DecisionTreeClassifier(random_state=42)
neural_network_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
neural_network_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# --- Model Training and Evaluation ---
print("Original Data:")
original_results = {}
original_results['KNN'] = train_and_evaluate_model(X_train, y_train, X_test, y_test, "KNN", knn_model)
original_results['Decision Tree'] = train_and_evaluate_model(X_train, y_train, X_test, y_test, "Decision Tree", decision_tree_model)
original_results['Neural Network'] = train_and_evaluate_model(X_train, y_train, X_test, y_test, "Neural Network", neural_network_model)

print("\nUndersampled Data:")
undersampled_results = {}
undersampled_results['KNN'] = train_and_evaluate_model(X_train_under, y_train_under, X_test_under, y_test_under, "KNN", knn_model)
undersampled_results['Decision Tree'] = train_and_evaluate_model(X_train_under, y_train_under, X_test_under, y_test_under, "Decision Tree", decision_tree_model)
undersampled_results['Neural Network'] = train_and_evaluate_model(X_train_under, y_train_under, X_test_under, y_test_under, "Neural Network", neural_network_model)


print("\nOversampled Data:")
oversampled_results = {}
oversampled_results['KNN'] = train_and_evaluate_model(X_train_over, y_train_over, X_test_over, y_test_over, "KNN", knn_model)
oversampled_results['Decision Tree'] = train_and_evaluate_model(X_train_over, y_train_over, X_test_over, y_test_over, "Decision Tree", decision_tree_model)
oversampled_results['Neural Network'] = train_and_evaluate_model(X_train_over, y_train_over, X_test_over, y_test_over, "Neural Network", neural_network_model)

import matplotlib.pyplot as plt
import numpy as np

# Assuming you have the original_results, undersampled_results, and oversampled_results dictionaries

# Extract accuracies for each model and sampling technique
knn_accuracies = [original_results['KNN'][0], undersampled_results['KNN'][0], oversampled_results['KNN'][0]]
dt_accuracies = [original_results['Decision Tree'][0], undersampled_results['Decision Tree'][0], oversampled_results['Decision Tree'][0]]
nn_accuracies = [original_results['Neural Network'][0], undersampled_results['Neural Network'][0], oversampled_results['Neural Network'][0]]

# Set up the bar chart
models = ['KNN', 'Decision Tree', 'Neural Network']
sampling_techniques = ['Original', 'Undersampled', 'Oversampled']
x = np.arange(len(models))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width, knn_accuracies, width, label='KNN')
rects2 = ax.bar(x, dt_accuracies, width, label='Decision Tree')
rects3 = ax.bar(x + width, nn_accuracies, width, label='Neural Network')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy Comparison across Sampling Techniques')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# Add value labels to the bars
def autolabel(rects):
    """Attach a text label above each bar in rects, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{:.2f}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()
plt.show()

pd.Series(y_resampled).value_counts()

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# ... (previous code for model training and evaluation) ...

# 1. Bar chart for accuracy
models = ['KNN', 'Decision Tree', 'Neural Network']
accuracies = [knn_test_accuracy, dt_test_accuracy, nn_test_accuracy]

plt.figure(figsize=(8, 6))
plt.bar(models, accuracies, color=['blue', 'green', 'red'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)  # Set y-axis limits for better visualization
plt.show()

# 2. Precision-Recall Comparison
plt.figure(figsize=(8, 6))
plt.bar(models, [knn_precision, dt_precision, nn_precision], color=['blue', 'green', 'red'], label='Precision')
plt.bar(models, [knn_recall, dt_recall, nn_recall], color=['lightblue', 'lightgreen', 'lightcoral'], label='Recall', alpha=0.7)  # Added alpha for transparency
plt.title('Precision-Recall Comparison')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.legend()
plt.show()

# 3. Confusion Matrices
# KNN Confusion Matrix
knn_cm = confusion_matrix(y_test, knn_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(knn_cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('KNN Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Decision Tree Confusion Matrix
dt_cm = confusion_matrix(y_test, dt_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(dt_cm, annot=True, fmt='d', cmap='Greens', cbar=False)
plt.title('Decision Tree Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Neural Network Confusion Matrix
nn_cm = confusion_matrix(y_test, nn_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(nn_cm, annot=True, fmt='d', cmap='Reds', cbar=False)
plt.title('Neural Network Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)